In [1]:
import moldesign as mdt
from moldesign import units as u
import ipywidgets as ipy

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med
# mdt.configure()

In [3]:
mdt.Atom


moldesign.molecules.atoms.Atom

In [4]:
mol = mdt.from_pdb('1yu8')

In [5]:
mol = mdt.Molecule(mol.residues[0:5])

In [6]:
mdmol = mdt.assign_forcefield(mol)
atom = mdmol.atoms[13]
print "Done parameterizing"

Connecting to docker host at unix://var/run/docker.sock... done


Widget Javascript not detected.  It may not be installed or enabled properly.


Done parameterizing


In [7]:
print mdmol.write('pdb')

ATOM      1  N   LEU 1   1      10.367  29.903  18.753  0.00  0.00           N  
ATOM      2  H   LEU 1   1       9.608  30.434  19.156  0.00  0.00           H  
ATOM      3  H2  LEU 1   1      10.313  29.972  17.747  0.00  0.00           H  
ATOM      4  H3  LEU 1   1      11.223  30.147  19.230  0.00  0.00           H  
ATOM      5  CA  LEU 1   1      10.139  28.458  19.063  0.00  0.00           C  
ATOM      6  HA  LEU 1   1      10.058  27.884  18.140  0.00  0.00           H  
ATOM      7  CB  LEU 1   1      11.320  27.909  19.881  0.00  0.00           C  
ATOM      8  HB2 LEU 1   1      12.162  28.232  19.268  0.00  0.00           H  
ATOM      9  HB3 LEU 1   1      11.307  28.480  20.809  0.00  0.00           H  
ATOM     10  CG  LEU 1   1      11.465  26.413  20.202  0.00  0.00           C  
ATOM     11  HG  LEU 1   1      12.445  26.239  20.647  0.00  0.00           H  
ATOM     12  CD1 LEU 1   1      10.381  25.970  21.174  0.00  0.00           C  
ATOM     13 HD11 LEU 1   1  

In [8]:
rs = mdt.widgets.ResidueSelector(mdmol)
rs

In [9]:
# mdmol.write('pdb')

In [10]:
canvas = mdmol.draw()

In [11]:
mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[57])
mdmol.calculate()
mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
                     temperature=300.0*u.kelvin, frame_interval=40.0*u.fs)

LAMMPS output is captured by PyLammps wrapper
LAMMPS Description

85 atoms
86 bonds
157 angles
364 dihedrals
0 impropers

11 atom types
30 bond types
65 angle types
84 dihedral types

-50.8505723 62.4447601 xlo xhi
-32.4876961 80.4339133 ylo yhi
-34.146 77.2376008 zlo zhi

Masses

1 14.01
2 1.008
3 12.01
4 1.008
5 1.008
6 12.01
7 16.0
8 1.008
9 16.0
10 1.008
11 1.008

Pair Coeffs

1 0.169999999918 3.24999852403 0.169999999918 3.24999852403
2 0.0157000000042 1.06907846172 0.0157000000042 1.06907846172
3 0.109399999916 3.39966950845 0.109399999916 3.39966950845
4 0.0157000000627 1.95997717901 0.0157000000627 1.95997717901
5 0.0157000000262 2.64953278726 0.0157000000262 2.64953278726
6 0.0860000001284 3.39966950794 0.0860000001284 3.39966950794
7 0.209999999842 2.95992190164 0.209999999842 2.95992190164
8 0.0157000000985 2.47135304264 0.0157000000985 2.47135304264
9 0.210400000249 3.06647338746 0.210400000249 3.06647338746
10 0 0.0 0 0.0
11 0.0150000000642 2.59964245874 0.0150000000642 2.

In [15]:
def apply_user_force(x, y, z):
    vec = [x, y, z]
    if all(v==0.0 for v in vec):
        return;
    vec = vec * u.angstrom
    mdmol.energy_model.apply_force(vec)
    mytraj = mdmol.run(200*u.fs)
    
    for step in mytraj:
        mdmol.energy_model.render_molecule(traj=step)
        canvas.set_positions(step.positions.value_in(u.angstrom))
#     for step in xrange(5):
#         mdmol.run(50 * u.fs)
#         mdmol.energy_model.render_molecule()
#         canvas.set_positions()
    
    mdmol.energy_model.finish_interaction()
    canvas.set_positions()
    
#     mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[657])
#     mdmol.calculate()

In [16]:
canvas.set_positions()
canvas.stick()
canvas

In [17]:
from ipywidgets import interact, interactive, fixed
interact(apply_user_force, x=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False),
            y=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False),
            z=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False))

<function __main__.apply_user_force>